In [1]:
from google.colab import drive
drive.mount('/content/drive')


import os
import tensorflow as tf
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping

base_path = '/content/drive/MyDrive/dataset'
train_dir = os.path.join(base_path, 'train')
test_dir = os.path.join(base_path, 'test')

def verify_structure(path):
    classes = ['covid', 'normal']
    for cls in classes:
        class_path = os.path.join(path, cls)
        if not os.path.exists(class_path):
            raise FileNotFoundError(f"Missing class folder: {class_path}")
        num_files = len(os.listdir(class_path))
        print(f"{cls} images: {num_files}")

print("Training set:")
verify_structure(train_dir)
print("\nTest set:")
verify_structure(test_dir)

train_generator = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    shuffle=True,
).prefetch(tf.data.AUTOTUNE)

test_generator = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=16,
    label_mode='binary',
    shuffle=False,
).prefetch(tf.data.AUTOTUNE)

base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3),
    pooling='avg'
)

base_model.trainable = True
for layer in base_model.layers[:150]:
    layer.trainable = False

model = tf.keras.Sequential([
    base_model,
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
optimizer = mixed_precision.LossScaleOptimizer(optimizer)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[EarlyStopping(patience=2)]
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"\nFinal Test Accuracy: {test_acc:.2%}")

Mounted at /content/drive
Training set:
covid images: 111
normal images: 112

Test set:
covid images: 28
normal images: 28
Found 223 files belonging to 2 classes.
Found 56 files belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 126s 14s/step - accuracy: 0.8289 - loss: 0.2479 - val_accuracy: 1.0000 - val_loss: 1.1921e-07
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 34s 692ms/step - accuracy: 1.0000 - loss: 4.7684e-04 - val_accuracy: 1.0000 - val_loss: 1.1921e-07
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 842ms/step - accuracy: 1.0000 - loss: 4.6748e-05 - val_accuracy: 1.0000 - val_loss: 1.1921e-07
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step - accuracy: 1.0000 - loss: 1.1921e-07

Final Test Accuracy: 100.00%
